In [14]:
from random import shuffle
import glob, h5py, pickle, cv2, re
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
width = 128
height = 128
channels = 3

sub = 'est_projected'
filename = 'datasets/'+sub+'.pkl'
print('File to be created: ' + filename)

File to be created: datasets/est_projected.pkl


In [15]:
path = '../data/est_projected_microDoppler/*_proj*.png'
files1 = glob.glob(path)
print(len(files1), 'files')
ang_tol = [5, 10, 15, 20, 25, 30, 35, 40, 45]
acc1 = []

for t in ang_tol:
    correct = 0
    wrong = 0
    train_labels = []
    train_images = []
    cnt = 0
    
    for i in range(len(files1)):
        fname = files1[i].split('/')[-1]
        if '\\' in fname:
            fname = fname.split('\\')[-1]
        class_idx = [m.end() for m in re.finditer('class', fname)]
        underscore_idx = [m.start() for m in re.finditer('_', fname)]
        classes = np.array([int(c) for c in fname[class_idx[0]: underscore_idx[1]].split('-')])

        angles = fname.split('_')[2][3:]
        plusses = angles.rfind('+')
        minuses = angles.rfind('-')

        if plusses == -1 and minuses == -1:
            all_ang_ids = int(angles)
            angs = all_ang_ids
        elif plusses == -1 and minuses != -1:
            all_ang_ids = sorted([minuses, len(angles)+1])
        elif plusses != -1 and minuses == -1:
            all_ang_ids = sorted([plusses, len(angles)+1])
        else:
            all_ang_ids = sorted([plusses, minuses, len(angles)+1])

        if isinstance(all_ang_ids, list):
            if all_ang_ids[0] == 0:
                angs = np.zeros((len(all_ang_ids) - 1,), dtype=int)
                for x in range(len(all_ang_ids) - 1):
                    angs[x] = int(angles[all_ang_ids[x]:all_ang_ids[x+1]]) # .astype(int)
            else:
                angs = np.zeros((len(all_ang_ids),), dtype=int)
                angs[0] = int(angles[:all_ang_ids[0]])
                for x in range(len(all_ang_ids) - 1):
                    angs[x+1] = int(angles[all_ang_ids[x]:all_ang_ids[x+1]]) # .astype(int)
        proj_ang = int(fname[fname.find('proj')+4: -4])
        
        for idx, a in enumerate(angs):
            if abs(proj_ang - a) < t:
                correct += 1
                clas = classes[idx]
                
                if clas == 13:
                    clas = 8
                elif clas == 14:
                    clas = 9
                    
                train_labels.append(clas)
                img = cv2.imread(files1[i])
                img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
                clear_output(wait=True)
                print('Ang_tol: '+ str(t) + ', Loading image '+str(cnt)+'/'+str(len(files1))+ ' ' + fname)
                train_images.append(img)
                cnt += 1
                break
            else:
                wrong += 1
                
    print(np.unique(train_labels))
    train_labels = to_categorical(np.array(train_labels))
    train_images = np.swapaxes(np.array(train_images), 1, 2).reshape(len(train_images), width, height, channels)/255.
    print(train_images.shape)
    print(train_labels.shape)
    acc1.append(correct/(correct + wrong) * 100)
    
    filename2 = filename[:-4] + '_angtol' + str(t) + '.'
    f = h5py.File(filename2, mode='w')
    f.create_dataset("train_img", train_images.shape, np.uint8)
    # f.create_dataset("test_img", test_images.shape, np.uint8)  
    f.create_dataset("train_labels", train_labels.shape, np.uint8)
    # f.create_dataset("test_labels", test_labels.shape, np.uint8)

    f["train_img"][...] = train_images
    # f["test_img"][...] = test_images
    f["train_labels"][...] = train_labels
    # f["test_labels"][...] = test_labels
    f.close()
    print(filename2 + ' created.')

Ang_tol: 45, Loading image 56/57 pCemreRony_class6-1_ang0+45_iter4_proj47.png
[1 2 6]
(57, 128, 128, 3)
(57, 7)
datasets/est_projected_angtol45. created.
